In [ ]:
import numpy as np
import pandas as pd
np.random.seed(0)
import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,RepeatVector
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
import tensorflow as tf
from keras import backend as K
np.random.seed(1)

#Fetching data

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/my projects/chatbots/Retrieval_bots/covid_19 chatbot/data/WHO_FAQ.csv")

In [ ]:
data.head()

,Context,Answer
0,What is a coronavirus?,Coronaviruses are a large family of viruses wh...
1,What is a coronavirus?,"In humans, several coronaviruses are known to ..."
2,What is COVID-19?,COVID-19 is the infectious disease caused by t...
3,What are the symptoms of COVID-19?,The most common symptoms of COVID-19 are fever...
4,What are the symptoms of COVID-19?,Some people become infected but don’t develop ...


In [ ]:
data.tail()

,Context,Answer
80,Are smokers and tobacco users at higher risk o...,Smokers are likely to be more vulnerable to CO...
81,Are smokers and tobacco users at higher risk o...,Smoking products such as water pipes often inv...
82,How large does a meeting or event need to be i...,High profile international sporting events suc...
83,How large does a meeting or event need to be i...,An event counts as a “mass gatherings” if the ...
84,Does WHO recommend that all international mass...,No. As each international mass gathering is di...


In [ ]:
questions=list(data["Context"])
answers=list(data["Answer"])

#**pre-processing data**

In [ ]:
import nltk
nltk.download('punkt')
import string
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
lemmer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
X = preprocessing(questions)

#**Building a model for similarity**

In [ ]:
!pip install sentence-transformers 

     |████████████████████████████████| 81kB 6.8MB/s 
     |████████████████████████████████| 2.1MB 23.9MB/s 
     |████████████████████████████████| 1.2MB 56.1MB/s 
     |████████████████████████████████| 3.3MB 52.8MB/s 
     |████████████████████████████████| 870kB 50.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=7758eed02e675337e0edd2889f4155a367ecf80291187dcfcee40e888ab84978
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=ef74faf47e7afda4be8ec34e878e56273129a8bdf34890690113f9bfe3326ede
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [ ]:

from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
# Sample for how to do similarity
test_sent = "what is coronavirus"
test_sent = LemNormalize(test_sent)
test_sent = " ".join(test_sent)
test_vec = sbert_model.encode([test_sent])[0]
query_vec = sbert_model.encode([" ".join(X[0])])[0]
sim = cosine(test_vec,query_vec)
print(sim)

0.9879526


#**Buiding a final chatbot**

In [ ]:
def calc_highest_sim(question,X,model):
  question = LemNormalize(question)
  ques_vec = model.encode([" ".join(question)])[0]
  answer_sim=0
  idx=-1
  sim=0
  for count,sentence in enumerate(X):
    temp=cosine(ques_vec,model.encode([" ".join(sentence)])[0])
    if temp > sim:
      sim=temp
      idx=count
  
  if sim < 0.4:
    idx=-1
  return idx

In [ ]:
print("HELLO ! I am a Covid 19 chatbot.Enter your question below and type \'end\' to exit")
while(True):
  question = input("Question : ")
  if(question == "end" or len(question)==0):
    print("Hope you got satisfactory answers. Thank You!")
    break
  else:
    idx = calc_highest_sim(question,X,sbert_model)

    if(idx == -1):
      print("BOT : Sorry I am not able to answer this question right now")
    else:
      print(f"BOT : {answers[idx]}")

HELLO ! I am a Covid 19 chatbot.Enter your question below and type 'end' to exit
Question : what is corona>
BOT : Coronaviruses are a large family of viruses which may cause illness in animals or humans. 
Question : are animal affected by it
BOT : Coronaviruses are a large family of viruses that are common in animals. Occasionally, people get infected with these viruses which may then spread to other people.
Question : can woman do breastfeed
BOT : Yes. Women with COVID-19 can breastfeed if they wish to do so.
Question : should i stay home?
BOT : Illness due to COVID-19 infection is generally mild, especially for children and young adults. However, it can cause serious illness: about 1 in every 5 people who catch it need hospital care.
Question : home
BOT : COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019.
Question : can a newborn child affected 
BOT